In [2]:
import pandas as pd
import numpy as np
import altair as alt
from IPython.display import HTML, Markdown
from jinja2 import Template, Environment, FileSystemLoader
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight') # bmh, ggplot, from plt.style.avalable
plt.rcParams['font.sans-serif']=['SimHei']  #解决中文显示乱码问题
plt.rcParams['axes.unicode_minus']=False  #解决坐标轴负数的负号显示问题

In [78]:
class Anaindex:
    def __init__(self, idx) -> None:
        '''
        github中的数据已经整理过，df只包括至2022年度数据，code去除了2023年上市公司的数据。

        '''
        self.idx = idx
        idx_s = f'../../mydata/yjbg_y2022_{idx}.pkl.gz'
        code_s = f'../../mydata//ashare_basic.pkl.gz'
        self.df = pd.read_pickle(idx_s)
        self.code = pd.read_pickle(code_s)
        self.df = self.df.replace(0, np.nan)
        self.df['year'] = self.df['qs'].str[0:4]
        self.quant = self.df.groupby('year')[idx].agg(['sum', 'count', 'mean', 'median', 'idxmax', 'idxmin']).reset_index()
        self.merged = self.df.merge(self.code)
        self.yeargp = self.merged.groupby('year')
        self.indudf = self.merged.groupby(['year','indu'], as_index=False)[self.idx].sum()
        self.indutb = self.indudf.pivot_table(index='indu', columns='year', values=self.idx, aggfunc='mean')
        self.namedf = self.merged.groupby(['year', 'name'], as_index=False)[self.idx].sum()
        self.nametb = self.namedf.pivot_table(index='name', columns='year', values=self.idx, aggfunc='mean')

    @staticmethod
    def showProfile(df) :
        col_profile = pd.concat([df.dtypes, df.isna().sum()/len(df)*100, df.sample().T], axis=1 ).reset_index()
        col_profile.columns = ['列名', '类型', '缺失比%','样例']
        print(df.shape)
        return (col_profile)
    
    def data_overall(self):
        
        def stat_cards():  
            sum_byear = self.quant['sum']                
            data = {
            'sum' : f'{sum_byear.sum():.3}',
            'inc' : f'{pow(sum_byear.iloc[-1]/sum_byear.iloc[0], 1/(len(sum_byear)-1)) - 1 : .2%}'}
            return self.card(data)
        
        # 每年的'sum, mean, median'
        quant_byear = {k:self.plot_ybar(self.quant[['year', k]], f'{k} of {self.idx}') for k in ['sum', 'mean', 'median']}
        # 每年的交易所，板块分布
        market_ex = {
            k:self.plot_market_ex(k)
            for k in ['market', 'ex']}

        res = {'stat': stat_cards()
               , 'quant_byear' : quant_byear
               , 'market_ex': market_ex
               }
        return res
    
    def card(self, data):
        '''
        输入字典类型数据
        输出：k是标题，v是内容一般为了显示巨大数据
        '''
        env = Environment(loader=FileSystemLoader('../_static/'))
        temp = env.get_template('test.html')
        return HTML(temp.render(dic_data = data))

    def plot_hbar(self, df, title, fmt='.2f'):
        '''
        基础hbar
        输入df必须规范化好，第一列标签是Y轴，第二列是数据X轴。
        输出为降排列的hbar
        '''

        cols=list(df.columns)
        fig = alt.Chart(df).mark_bar().encode(
        alt.X(f'{cols[1]}:Q').axis(format=fmt),
        alt.Y(f'{cols[0]}:O').sort('-x'),
        tooltip=cols
        ).properties(
            title = title,
            width = 'container',
            height = len(df)*18+80
        )
        return fig
    
    def plot_ybar(self, df, title, fmt='.2s'):
        '''
        基础hbar
        输入df必须规范化好，第一列标签是Y轴年份，第二列是数据X轴。
        输出为按年排列的bar
        '''

        cols=list(df.columns)
        fig = alt.Chart(df).mark_bar().encode(
        alt.X(f'{cols[1]}:Q').axis(format=fmt),
        alt.Y(f'{cols[0]}:O'),
        tooltip=cols
        ).properties(
            title = title,
            width = 'container',
            height = len(df)*18+80
        )
        return fig
    
    def plot_min(self):
        mindf = self.df.loc[self.quant['idxmin'],['year','code', self.idx]].reset_index(drop = True)
        mindf = mindf.merge(self.code[['code', 'name']])
        base = self.plot_ybar(mindf[['year', self.idx, 'name']],f'min of {self.idx} by year')
        fig = base.mark_bar(color='brown') + base.mark_text(align='left', color='white').encode(text='name')
        return fig
    
    def plot_max(self):
        maxdf = self.df.loc[self.quant['idxmax'],['year','code', self.idx]].reset_index(drop = True)
        maxdf = maxdf.merge(self.code[['code', 'name']])
        base = self.plot_ybar(maxdf[['year', self.idx, 'name']],f'max of {self.idx} by year')
        fig = base.mark_bar() + base.mark_text(align='right', color='white')
        return fig
    
    def plot_bin(self, df, title='分布', f='.0f'):
        # 第一列是属性，对第二列bin
        
        fig = alt.Chart(df, title = title, width='container').mark_bar().encode(
        alt.Y(f'{df.columns[1]}:Q').bin().axis(format=f),
        alt.X('count()').axis(title='数量'))
        return fig
        
    def plot_sbar(self, df):
        '''
        输出为：纵轴按年份排列，横轴按类别进行百分比对比的条形图。
        数据第一行为年份，第二列为分类，第3列为数据。
        '''
        cols = df.columns
        fig = alt.Chart(df).mark_bar().encode(
            alt.X(f'sum({cols[2]})').stack('normalize'),
            alt.Y(f'{cols[0]}:O'),
            color= cols[1]
        ).properties(
            title = f'percent by {cols[1]}',
            width = 'container',
            height = len(df)*9+60
        )
        return fig
    
    def plot_lose_compare(self, df):
        '''
        输入三列：year, cat, data
        输出按年排列，正负值的数量和金额对比两张图。
        '''
        cols = df.columns
        ldf = df[df[cols[2]]<0].groupby('year')[cols[2]].agg(
            count='count',
            sum = lambda x: np.abs(x.sum()),
            正负 = lambda x: '负值'
        )
        wdf = df[df[cols[2]]>0].groupby('year')[cols[2]].agg(
            count='count',
            sum = lambda x: np.abs(x.sum()),
            正负 = lambda x: '正值'
        )
        data = pd.concat([ldf, wdf]).reset_index()
        res={
            'count': self.plot_sbar(data[['year', '正负', 'count']]),
            'sum': self.plot_sbar(data[['year', '正负', 'sum']])
        }

        return res

    def plot_market_ex(self, cat):
        '''
        按市场和按交易所的变化可以合成一组。
        cat:market, ex
        '''
        df  = profit.merged.groupby(['year',cat])[self.idx].sum().to_frame(self.idx).reset_index()
        fig = self.plot_sbar(df)

        return fig

    def mostn(self, df, n=10):
        '''
        输入为3列df,第一列为year, indu | name, data
        mostn: 合计前后n
        mostn_byear：分年前后n
        mostn_count:上榜前后n次数统计
        '''
        res = {}
        for flag in [0, 1]:
            res[f'mostn_{flag}'] = self.plot_mostn( df, flag)
            res[f'mostn_byear_{flag}'] = self.plot_mostn_byear( df, flag)
            res[f'mostn_count_{flag}'] = self.plot_mostn_count( df, flag)
            res[f'most_{"min" if flag else "max"}'] = self.plot_minmax_years(df, flag)

        return res

    def plot_mostn(self, df, flag, agg='sum', n=10, f='.0s'):
        '''
        输入为3列df,第一列为year, indu | name, data
        flag=0, 统计总和前十的行业。不分年。
        flag=1, 统计总和后十的行业。不分年。        
        '''
        cols = df.columns
        df = df.groupby(cols[1])[cols[2]].agg(agg).sort_values(ascending=flag).head(n).to_frame().reset_index()
        title = f'{cols[1]}总{cols[2]}后十' if flag else f'{cols[1]}总{cols[2]}前{n}'
        fig = self.plot_hbar( df, title, f)

        return fig

    def plot_mostn_count(self, df, flag, n=10):
        '''
        df columns : year, indu, profit
        统计每一年份进入前后十名的名字，然后统计出现次数
        flag = 0, 前十汇总
        flag = 1, 后十汇总
        n为前10
        '''
        cols = df.columns
        topn = df.sort_values(by=[cols[0], cols[2]], ascending=[True, flag]).groupby(cols[0], as_index=False).nth[:n][cols[1]].value_counts().to_frame('count').reset_index()

        title = f'{cols[1]}前{n}上榜次数统计[{len(topn)}个]' if flag == 0 else f'{cols[1]}后{n}上榜次数统计[{len(topn)}个]'

        fig = self.plot_hbar(topn, title, '.0f')
        return fig

    def plot_minmax_years(self, df, flag, f='.0s'):
        cols = df.columns
        top = (df.sort_values(by=[cols[0], cols[2]], ascending=[True, flag])
               .groupby(cols[0], as_index=False)
               .nth[0])
        top[cols[0]] = top[cols[0]] + '-' + top[cols[1]]
        title = f'{cols[1]}历年{cols[2]}最小值' if flag else f'{cols[1]}历年{cols[2]}最大值'
        fig = self.plot_ybar(top[[cols[0], cols[2]]], title, fmt=f)
        return fig

    def plot_mostn_byear(self, df, flag, n=10):
        '''
        df columns : year, indu, profit
        flag=0, 统计每年利润前十的名字，3年一组全部绘制出来。
        flag=1, 统计每年利润后十的名字，3年一组全部绘制出来。
        '''
        cols = df.columns

        df = df.sort_values(by=[cols[0], cols[2]], ascending=[True, flag]).groupby(cols[0], as_index=False).nth[:n]
        # 统一x坐标，以显示历史变化。
        smax = df[cols[2]].max()*1.05
        smin = df[cols[2]].min()*1.05

        gp = df.groupby(cols[0], as_index=False)

        # 绘制每一年的df图，存入list
        chartlist = []
        sort = 'x' if flag else '-x'
        for a, b in gp:
            fig = alt.Chart(b).mark_bar(color= 'brown' if flag else 'cornflowerblue').encode(
            alt.X(f'{cols[2]}:Q').axis(format='.0s').scale(domain=(smin if flag else 0,smax)),
            alt.Y(f'{cols[1]}:O').sort(sort),
            ).properties(
                title = a,
                width = 180,
                height = n*16+60
            )
            chartlist.append(fig)

        # 一行两列图形，先分割成两个一组的列表
        chars = [chartlist[i:i+3] for i in range(0, len(chartlist), 3)]
        # 最终的图形是纵向拼接每一组两个横向拼接的图
        figs = alt.vconcat()
        for r in chars:
            row = alt.hconcat()
            for c in r:
                row |= c
            figs &= row
        
        return figs

    def plot_den(self, s):
        '''
        input is a series
        将s从大到小排列，然后累计占比。
        输出一个累计百分比图，显示集中度。
        横轴是排名前百分之几的项目，纵轴是累计占比。
        由天alt只能接受行数少于5000, 所以将数据百分比化。
        200行内直接给制，200行以上将数据缩减到100个。
        '''
        s = s.sort_values(ascending=0)
        step = int(len(s)/100) if len(s) > 200 else 1
        cum_pct = (100*s/s.sum()).cumsum()
        num_pct = np.array([i+1 for i in range(len(s))]) *100 / len(s)
        df = pd.DataFrame({'cum_pct':cum_pct, 'num_pct':num_pct})
        df = df.iloc[::step,:]
        fig = alt.Chart(df).mark_line().encode(
        y='cum_pct',
        x='num_pct').properties(
            width = 'container'
        )
        return fig

    def plot_bank_pct(self):
        '''
        银行、保险、券商三个行业加一起的利润占比的按年变化。
        '''
        df = self.merged
        df['cat'] = df['indu'].apply(lambda x: 'banks' if x in ['银行','保险','证券'] else 'others')
        banks_profit = df.groupby(['year','cat'])[['profit']].sum().reset_index()
        
        fig = alt.Chart(banks_profit).mark_bar().encode(
            alt.X(f'sum({self.idx})').stack('normalize'),
            alt.Y('year:O'),
            color= 'cat'
        ).properties(
            title = '大金融利润占比变化',
            width = 'container',
            height = 340
        )
        return fig

    def code_shapee(self):
        '''
        计算所有代码的shapee值
        输出一个从历年到最后一个报告年度的df
        以及一个平均值的s
        '''
        indu_table = self.merged.pivot_table(index='code', columns='year', values=self.idx, aggfunc='sum')
        cols = indu_table.columns

        res_df = pd.DataFrame()

        for i in range(0, len(cols)-5):

            # 以每年的开始持有到目前都做测试。到前三年止，只计算两年的数据可能会奇高。
            sel = indu_table.iloc[:,i:]
            # 去掉所有包含空值的项目，空值表明要么当时还没有上市，要么表示当年没法出报告。
            sel = sel.dropna(axis=0)

            # 出现过亏损的去掉，因为无法正常计算平均增长率
            sel = sel[sel.min(axis=1) >0].pct_change(axis=1)
            res = sel.mean(axis=1) / sel.std(axis=1)
            res = res.sort_values(ascending=0).to_frame('shapee').reset_index()
            res['year'] = cols[i]
            res_df = pd.concat([res_df, res])

        df_by_year = res_df.pivot_table(index='code', columns='year', values='shapee').sort_values(by='2010', ascending=0)
        s_avg = df_by_year.mean(axis=1).sort_values(ascending=0)

        return (df_by_year, s_avg)

    def indu_shapee(self):
        '''
        计算所有行业的shapee值
        输出一个从历年到最后一个报告年度的df
        以及一个平均值的s
        '''
        indu_table = self.merged.pivot_table(index=['code','indu'], columns='year', values=self.idx, aggfunc='sum')
        cols = indu_table.columns

        res_df = pd.DataFrame()

        for i in range(0, len(cols)-3):

            # 以每年的开始持有到目前都做测试。到前三年止，只计算两年的数据可能会奇高。
            sel = indu_table.iloc[:,i:]
            # 去掉所有包含空值的项目，空值表明要么当时还没有这个行业，要么表示当年没法出报告。
            sel = sel.dropna(axis=0)
            # group到indu
            byindu = sel.groupby(level=1).mean()
            # 行业出现过亏损的去掉，因为无法正常计算平均增长率
            byindu = byindu[byindu.min(axis=1) >0].pct_change(axis=1)
            res = byindu.mean(axis=1) / byindu.std(axis=1)
            res = res.sort_values(ascending=0).to_frame('shapee').reset_index()
            res['year'] = cols[i]
            res_df = pd.concat([res_df, res])

        df_by_year = res_df.pivot_table(index='indu', columns='year', values='shapee').sort_values(by='2010', ascending=0)
        s_avg = df_by_year.mean(axis=1).sort_values(ascending=0)

        return (df_by_year, s_avg)
        
    def inc_byear(self, df):
        '''
        df有三列
        df: merged[['year', 'name', 'profit']] or indudf
        '''
        df = df.groupby('year')[[self.idx]].sum().pct_change().reset_index()
        a_mean = {'平均增长':f'{df[self.idx].mean():.2%}'}
        return {'byear':self.plot_ybar(df, f'历年{self.idx}增长', '.0%'),'a_mean': self.card(a_mean)}

    def inc_data(self, df):
        '''
        输入一个表：col是年份，index是标签。
        '''
        wdf = df[df.min(1)>0] # 去除历史上出现过负值的行
        wdf = wdf.pct_change(axis=1).iloc[:, 1:] # 去除第1列，因为是空值
        all_win = wdf[wdf.min(1)>0]
        stat = {'总数':len(df),'保持正值':len(wdf),'保持增长': len(all_win)}

        neg_count = wdf[wdf < 0].count(axis=1).to_frame('次数').reset_index()
        # win_bin = win_bin.to_frame('数量').reset_index(names= '负增长次数')
        # plot_win_bin = self.plot_ybar(win_bin, '出现负增长情况统计','.0f')
        # count_gp = wdf[wdf < 0 ].count(axis=1).to_frame('counts').groupby('counts')
        # def plot_abin_byear(i, title):
        #     df = wdf[wdf.index.isin(count_gp.get_group(i).index) == True]
        #     df = df.melt(var_name='year', value_name='inc', ignore_index=False).reset_index()
        #     fig = alt.Chart(df).mark_line().encode(
        #         alt.X('year:O'),
        #         alt.Y('inc:Q'),
        #         alt.Color('indu').legend(orient='top')
        #     ).properties(
        #         title= title,
        #         width = 'container'
        #     )
        #     return fig

        def plot_bin(df=neg_count, title='负增长次数分布', f='.0f'):
            # 第一列是属性，对第二列bin
        
            fig = alt.Chart(df, title= title, width='container').mark_bar().encode(
            alt.Y(f'{df.columns[1]}:Q').bin().axis(format=f),
            alt.X('count()').axis(title='数量'))
            return fig
        
        # def plot_bin_most10(df, f, title):        
        #     # 数据分析做条形图
        #     # 取数据的前十和后十做条形图
        #     fig_bin = plot_bin(df, f'{title}分布', f)
        #     fig1 = self.plot_hbar(df.head(10), f'{title}前十', f)
        #     fig2 = self.plot_hbar(df.tail(10), f'{title}后十', f)
        #     return (fig_bin, fig1, fig2)
        
        # def plot_mean():
        #     df = wdf.mean(1).sort_values(ascending=0).to_frame('mean').reset_index()
        #     return plot_bin_most10(df, '.0%', '平均增长率')

        # def plot_shapee():
        #     df = (wdf.mean(1) / wdf.std(1)).sort_values(ascending = 0).to_frame('shapee').reset_index()
        #     return plot_bin_most10(df, '.2f', '夏普指数')

        return (wdf, neg_count, plot_bin())

    def get_inc(self, df, agg='sum'):
        '''
        输入三列：年份，标签，数据，数据去除了绝对值为负的项目。
        输入：同样三列，将原来的数据列变化增长率
        '''
        cols=df.columns
        dft = df.pivot_table(index=cols[0], columns=cols[1], values=cols[2], aggfunc=agg)
        dft = dft.loc[:,dft.min() > 0]
        dft = np.log1p(dft.pct_change()).iloc[1:,:]
        ldf = dft.melt(var_name=cols[1], value_name='inc', ignore_index=False).reset_index()
        return ldf

    def get_shapee(self, df):
        '''
        输入三列：year, indu|name, inc,即通过get_inc()轮换的df。
        输出三列：year, indu|name, shapee
        '''
        df = self.get_inc(df)  
        cols = df.columns      
        dft = df.pivot_table(index=cols[0], columns=cols[1], values=cols[2], aggfunc='sum')
        res_df = pd.DataFrame()
        for i in range(len(dft) -3 ):
            res = (dft.iloc[i:,].mean() / dft.iloc[i:,].std()).to_frame('shapee')
            res['year'] = dft.index[i]
            res_df = pd.concat([res_df,res])

        res_df = res_df.reset_index()[['year', cols[1], 'shapee']]

        return res_df
    
    def tidy_name(self, df):
        '''
        数据集中个股的报表数据出现多个空值。这分两种情况：
        1、报告首年不是从2010年开始的。
        2、在某一年缺失报告数。
        第二种情况出现说明出现了某种不好的状况，所以要将第二种情况排除掉。
        首先计算出报告首年度（注意和上市年度并不对应），第一个报告年度以前出现的空值没问题，以后出现的空值排除。
        每一个报告首年度对应一个报告总数，如果实际报告数等于这个数则保留，否则排除。
        '''

        cols = df.columns
        fyear = df.sort_values([cols[1], cols[0]]).groupby(cols[1]).nth(0)
        fyear['fyear'] = fyear['year']
        dft = df.pivot_table(index=cols[1], columns=cols[0], values=cols[2], aggfunc='sum')
        dft = dft.replace(0, np.nan)

        k = dft.columns
        v = [len(k)-i for i in range(len(k))]
        tdic = dict(zip(k,v))

        fyear['r_counts'] = fyear['fyear'].apply(lambda x : tdic[x])
        sel = fyear.merge(dft.count(axis=1).to_frame('r_act').reset_index(), on = cols[1])
        sel0 = sel[sel['r_counts'] == sel['r_act']][cols[1]]
        # 中间缺少数据的个股96家，大部分不重要，但有两个：中国移动和中国人保相对重要。此数据集质量一般。
        # dft_sel = dft.loc[sel0,]
        df_sel = df.replace(0, np.nan).loc[df['name'].isin(sel0),]

        return df_sel

    def plot_neg_pos_compare(self, df):
        '''
        输出正负值的数量和金额比
        输入为三年，年，标签，数据
        '''
        cols = df.columns
        c_win = df[df[cols[2]]> 0 ].groupby(cols[0])[[cols[1]]].count().reset_index()
        c_loss = df[df[cols[2]]< 0 ].groupby(cols[0])[[cols[1]]].count().reset_index()
        c_win['cat'] = '赢利'
        c_loss['cat'] = '亏损'
        cdf = pd.concat([c_win, c_loss], ignore_index=True)
        cdf.columns = [cols[0],cols[2], 'cat']
        cdf['typ'] = '数量比'

        s_win = df[df[cols[2]]> 0 ].groupby(cols[0])[[cols[2]]].sum().reset_index()
        s_loss = df[df[cols[2]]< 0 ].groupby(cols[0])[[cols[2]]].sum().multiply(-1).reset_index()
        s_win['cat'] = '赢利'
        s_loss['cat'] = '亏损'
        sdf = pd.concat([s_win, s_loss], ignore_index=True)
        sdf.columns = [cols[0],cols[2], 'cat']
        sdf['typ'] = '金额比'

        plot_df = pd.concat([cdf, sdf])

        fig = alt.Chart(plot_df).mark_bar().encode(
            alt.X(f'{cols[2]}:Q').stack('normalize').axis(title='百分比'),
            alt.Y('typ:O').axis(title=None),
            alt.Color('cat'),
            row=f'{cols[0]}'
        ).properties(
            title='正负值数量和金额对比',
            width = 600,  # width = container seems not work in any h or v stack
            # height = len(cdf),

        )
        return fig

profit = Anaindex('profit')

## 总体情况

- 总利润和总的利润增长（几何平均）

In [31]:
profit.data_overall()['stat']

In [32]:
profit.data_overall()['quant_byear']['sum']

alt.Chart(...)

- 上市公司总体利润保持增长,特别是疫情爆发后第2年,即2021年,出现了爆涨.
- 利润增长是公司利润增长和新增上市公司共同作用的结果。

In [71]:
profit.inc_byear(profit.indudf)['byear']

alt.Chart(...)

- 最近一年是利润增长最少的一年。

In [33]:
profit.data_overall()['quant_byear']['mean']

alt.Chart(...)

- 上市公司的平均利润总体有增长，大约为10亿左右。

In [34]:
profit.data_overall()['quant_byear']['median']

alt.Chart(...)

- 上市公司利润中位数在2022年急剧下降，略高于1亿。
- 平均值和中位数差一个数量级，说明上市公司利润的分布是左偏的。

In [35]:
profit.data_overall()['market_ex']['market']

alt.Chart(...)

In [36]:
profit.data_overall()['market_ex']['ex']

alt.Chart(...)

- 2020年对上交所打击比较大。
- 但第二年2021年，上交所又迎来狂欢。

In [79]:
profit.plot_neg_pos_compare(profit.namedf)

alt.Chart(...)

- 录得亏损的上市公司无论是数量还是金额都在持续增加。
- 相比之下，数量的增长更加明显。
- 2022年有超过20%的公司出现亏损。

In [ ]:
profit.plot_min('历年亏损王')

- 上市公司亏损王呈现出不断创新高的情况。
- 谁又能想到中海远控，曾经勇巨亏王呢？
- 疫情来临之际，中海远控大翻身，而同为交通行业的大航空，却整整苦了三年。
- 看到其中的一个亏损王，感觉自己也算是亲历一段历史。

In [ ]:
profit.plot_max('历年赢利王')

- 赢利王没什么惊喜，宇宙第一行，永远行，疫情完全没受到影响。可怕。

## 利润构成-分行业看

- 一般人可能不知道，中国上市银行有多挣钱，不然无法解释为什么银行股在大部分时候都被嫌弃。
- 先看一下所有利润加在一起按行业分的情况。

### 分行业利润绝对值

In [38]:
indus = profit.mostn(profit.indudf)

In [81]:
profit.plot_neg_pos_compare(profit.indudf)

alt.Chart(...)

In [84]:
profit.plot_den(profit.indudf.groupby('indu')['profit'].sum())

alt.Chart(...)

- 大约前20%的行业占了全部利润的80%
- 排名第一的行业占比超过了40%，很显然就是银行业。

In [39]:
indus['mostn_0']

alt.Chart(...)

- 这个数据极度集中，银行占据了超过43%的总利润。
- 前三个行业占比超过了50%， 前20个行业取得了80%的利润。真八二定律。

In [42]:
indus['mostn_1']

alt.Chart(...)

- 只有航空机场和教育行业是总体亏损了。
- 航空机场应该是受疫情影响。
- 教育行业算是团灭。

In [40]:
indus['mostn_count_0']

alt.Chart(...)

- 行业利润榜是指每年平均利润进入前十的行业，汇总统计资料。如果使用总利润的话，结果变化不大。
- 有三个行业年年都上榜，银行业年年都第一。
- 利润上榜的行业比较集中，只有18个行业。每年变化都不太大。

In [41]:
indus['mostn_count_1']

alt.Chart(...)

- 而在每年利润落后榜单中，数量就很大了。
- 86个行业中，有超过一半，46个行业，在过去13年中至少一次进入到利润倒数10名内。
- 教育行业，这个是没想到的，原来一直这么惨。
- 半导体也是，炒得这么厉害，结果，大部分时间利润很惨。
- 房地产开发，跌落神坛。

In [43]:
indus['most_max']

alt.Chart(...)

- 每年的利润最大行业都是银行，而且还在不断增长。
- 疫情期间的增长特别可观，为什么？

In [44]:
indus['most_min']

alt.Chart(...)

- 光伏行业、采掘行业、航空机场曾经两度最差。
- 疫情对航空机场的影响特别巨大。
- 十年前的光伏行业连续低迷，也算是见证到了。

In [45]:
indus['mostn_byear_0']

alt.VConcatChart(...)

In [46]:
indus['mostn_byear_1']

alt.VConcatChart(...)

- 房地产开发出现了巨大的由盛转衰，从利润第二、第三，快速到如今的倒数第二。

### 行业增长
- 数据是去除了出现过亏损的行业。出现过亏损无法正确计算增长率。
- 使用对数增长率，所以增长率可以相加。

In [54]:
indu_inc = profit.get_inc(profit.indudf)
indus_inc_mostn = profit.mostn(indu_inc)

In [55]:
indus_inc_mostn['mostn_0']

alt.Chart(...)

In [56]:
indus_inc_mostn['mostn_1']

alt.Chart(...)

- 环保行业，汽车整车，软件开发，这些感觉很火的行业，其实利润增长很有限。
- 旅游酒店行业的利润快速萎缩可能是暂时的，疫情过后应该能恢复。

In [57]:
indus_inc_mostn['most_max']

alt.Chart(...)

In [58]:
indus_inc_mostn['most_min']

alt.Chart(...)

- 航运港口，上窜下跳，经常上一年负增长最大，下一年又正增长最大。
- 医疗服务、互联网服务、文化传媒等多个行业也有类似情况。
- 也许这是一种周期性的模式？暴发性增长后，常伴随着报复性负增长。

In [59]:
indus_inc_mostn['mostn_count_0']

alt.Chart(...)

In [60]:
indus_inc_mostn['mostn_count_1']

alt.Chart(...)

- 这个方面重复性就很高了。
- 说明行业是充分分散化的，一个行业很难保持永远增长。
- 但实际上有两个行业是保持了期间内一直正增长。

In [61]:
indus_inc_mostn['mostn_byear_0']

alt.VConcatChart(...)

In [62]:
indus_inc_mostn['mostn_byear_1']

alt.VConcatChart(...)

- 2017年和2021年，看起来象是黄金时代。
- 2022年无疑是最惨的一年。

### 行业利润增长波动
- 去除了出现亏损的行业。
- 有的值是总和，只有排名上的意义。

In [64]:
indu_sha = profit.get_shapee(profit.indudf)
indus_sha = profit.mostn(indu_sha)

In [65]:
indus_sha['mostn_0']

alt.Chart(...)

In [66]:
indus_sha['mostn_1']

alt.Chart(...)

- 这个应该有比较强的参考性。
- 环保行业很奇怪。

In [67]:
indus_sha['mostn_count_0']

alt.Chart(...)

In [68]:
indus_sha['mostn_count_1']

alt.Chart(...)

In [69]:
indus_sha['mostn_byear_0']

alt.VConcatChart(...)

- 这个数据的含义是自某一年起持有到现在的shapee值。
- 所以只有横向比较的意义，纵向比肯定是时间越短波动越大。

In [70]:
indus_sha['mostn_byear_1']

alt.VConcatChart(...)

- 时间越短波动越大，这是可以理解的。shapee值最差的行业更明显。

## 利润增长

- 总体利润增长
- 行业利润增长
- 个股利润增长

- 利润增长包括了已经上市企业的业绩增长和新发上市带来的利润增长。
- 2022年在IPO数量创新高的情况下，利润增长几乎停滞，是个不好的迹象。

In [88]:
def lose_indu():
    df = profit.indudf
    ldf = df[df['profit'] < 0].sort_values(['year', 'profit'])

    count_sum_byyear = ldf.groupby('year').agg(
        数量=('profit', 'count'),
        金额=('profit', lambda x : x.sum()/1000000000)).reset_index().style.set_caption('历年亏损行业数量和金额(B)').format(precision=2)

    lose_detail_bycount = ldf.groupby('indu').agg(
        数量=('profit', 'count'),
        金额=('profit', lambda x : x.sum()/1000000000)).sort_values(by='数量', ascending=0).reset_index().style.set_caption('亏损行业次数和金额（B）').format(precision=2)
    
    overview = ldf.reset_index().pivot_table(index='indu', columns='year', values='profit', aggfunc='count', fill_value=0,margins=True).sort_values('All', ascending=0).style.set_caption('亏损行业次数一览表')
    overview_sum = ldf.reset_index().pivot_table(index='indu', columns='year', values='profit', aggfunc='sum', fill_value=0,margins=True).sort_values('All', ascending=1).div(1000000000).style.format(precision=2).set_caption('亏损行业金额一览表')

    return (count_sum_byyear, lose_detail_bycount, overview, overview_sum)


In [100]:
for i in lose_indu(): 
    display(i)

,year,数量,金额
0,2012,3,-9.96
1,2013,1,-2.69
2,2014,2,-8.05
3,2015,5,-61.02
4,2016,4,-27.82
5,2017,2,-8.69
6,2018,5,-8.68
7,2019,4,-50.68
8,2020,3,-115.98
9,2021,8,-147.29


,indu,数量,金额
0,船舶制造,5,-4.82
1,化肥行业,4,-48.27
2,教育,4,-12.22
3,有色金属,3,-9.53
4,航空机场,3,-289.57
5,光伏设备,2,-7.34
6,采掘行业,2,-30.59
7,装修装饰,2,-26.82
8,电机,2,-4.45
9,游戏,2,-3.21


year,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,All
indu,,,,,,,,,,,,
All,3,1,2,5,4,2,5,4,3,8,10,47
船舶制造,0,0,0,1,1,0,1,0,0,1,1,5
化肥行业,0,0,1,0,1,1,0,1,0,0,0,4
教育,0,0,0,0,0,0,1,0,1,1,1,4
航空机场,0,0,0,0,0,0,0,0,1,1,1,3
有色金属,1,0,1,0,0,0,1,0,0,0,0,3
光伏设备,1,1,0,0,0,0,0,0,0,0,0,2
钢铁行业,1,0,0,1,0,0,0,0,0,0,0,2
采掘行业,0,0,0,0,1,1,0,0,0,0,0,2


year,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,All
indu,,,,,,,,,,,,
All,-9.96,-2.69,-8.05,-61.02,-27.82,-8.69,-8.68,-50.68,-115.98,-147.29,-245.67,-686.53
航空机场,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-110.39,-37.96,-141.22,-289.57
钢铁行业,-4.55,0.00,0.00,-50.53,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-55.08
房地产开发,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-50.07,-50.07
化肥行业,0.00,0.00,-0.13,0.00,-3.04,-1.22,0.00,-43.88,0.00,0.00,0.00,-48.27
商业百货,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-30.26,-16.31,-46.57
农牧饲渔,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-43.18,0.00,-43.18
采掘行业,0.00,0.00,0.00,0.00,-23.12,-7.47,0.00,0.00,0.00,0.00,0.00,-30.59
装修装饰,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-18.73,-8.08,-26.82


- 2010年和2012年居然没有任何一个行业出现亏损，如日中天。
- 2021年起亏损行业的数量开始井喷，2022年亏损行业个数和金额都达到历史新高，且远超从前。
- 累计亏损金额最大的是航空机场，这个不奇怪，近三年的疫情打击惨重。
- 船舶制造亏损次数最多，重资产行业。
- 教育、房地产服务行业，明显受近期政策打压的影响很大。
- 光伏行业在十年前也曾经是亏损大户。
- 化肥行业也曾经历过较长时间的亏损。

In [90]:
def win_indu():
    df = profit.merged.groupby(['indu', 'year'], as_index=False)['profit'].mean()
    wdf = df.pivot_table(index='indu', columns='year', values='profit', aggfunc='sum')
    wdf = wdf[wdf.min(1)>0]
    wdf_inc = wdf.pct_change(axis=1).dropna(axis=1)
    all_win = wdf_inc[wdf_inc.min(1)>0]
    stat = {'行业总数':len(df),'保持赢利':len(wdf),'保持利润增长': len(all_win)}

    win_bin = wdf_inc[wdf_inc < 0].count(axis=1).value_counts().sort_index()
    win_bin = win_bin.to_frame('行业数').reset_index(names= '负增长次数')
    plot_win_bin = profit.plot_ybar(win_bin, '出现负增长情况统计','.0f')

    count_gp = wdf_inc[wdf_inc < 0 ].count(axis=1).to_frame('counts').groupby('counts')
    def plot_indu_year(i, title):
        df = wdf_inc[wdf_inc.index.isin(count_gp.get_group(i).index) == True]
        df = df.melt(var_name='year', value_name='inc', ignore_index=False).reset_index()
        fig = alt.Chart(df).mark_line().encode(
            alt.X('year:O'),
            alt.Y('inc:Q'),
            alt.Color('indu').legend(orient='top')
        ).properties(
            title= title,
            width = 'container'
        )
        return fig

    def plot_bin(df, title, f='.0f'):
        # 第一列是属性，对第二列bin
        fig = alt.Chart(df, title= title, width='container').mark_bar().encode(
        alt.Y(f'{df.columns[1]}:Q').bin().axis(format=f),
        alt.X('count()').axis(title='数量'))
        return fig
    
    def plot_bin_most10(df, f, title):        
        # 数据分析做条形图
        # 取数据的前十和后十做条形图
        fig_bin = plot_bin(df, f'{title}分布', f)
        fig1 = profit.plot_hbar(df.head(10), f'{title}前十', f)
        fig2 = profit.plot_hbar(df.tail(10), f'{title}后十', f)
        return (fig_bin, fig1, fig2)
    
    def plot_mean():
        df = wdf_inc.mean(1).sort_values(ascending=0).to_frame('mean').reset_index()
        return plot_bin_most10(df, '.0%', '平均增长率')

    def plot_shapee():
        df = (wdf_inc.mean(1) / wdf_inc.std(1)).sort_values(ascending = 0).to_frame('shapee').reset_index()
        return plot_bin_most10(df, '.2f', '夏普指数')

    return (plot_win_bin, plot_indu_year, wdf_inc, plot_mean(), plot_shapee())

indu_res = win_indu()


In [91]:

indu_res[0]

alt.Chart(...)

- 利润增长是用行业的利润总和还是平均利润呢？两次都试了一下，结果略有差异，但总体结论类似。
- 为了减少新上市对利润增长的影响，以下就以平均值来分析一下。
- 有一个行业从来没来出现过负增长，可以说是奇迹了。
- 有一个行业在12年内出现了8次负增长，就这样个情况它还没有出现整体亏损。
- 出现4到6次负增长的行业占了大多数。持续保持利润正增长是件不容易的事，也许以后会更难。

In [92]:
indu_res[1](0,'没有出现过负增长的行业')

alt.Chart(...)

- 银行业在2020年险些失身到到利润下降，但后两年神奇地回到了历史高位。

In [93]:
indu_res[1](1, '只出现过一次负增长的行业')

alt.Chart(...)

- 工程建设行业，2012年出现一次负增长，此后一直保持增长，但总趋势是下降的。

In [94]:
indu_res[1](2, '只出现过两次负增长的行业')

alt.Chart(...)

- 消费电子和医疗器械大起大落。
- 医药商业和酿酒行业比较平稳。

In [95]:
indu_res[1](8, '负增长次数最多的行业')

alt.Chart(...)

- 通用设备这个太妖了，以至于坐标轴都不好弄了，2020年利润涨了7倍多？
- 纺织服装乏力，应该是可以理解的。

In [96]:
indu_res[1](7, '出现7次负增长的行业')

alt.Chart(...)

- 石油行业，可以理解，总体肯定是向下走的。
- 有过两次超50%的利润下滑，也有在2021年利润翻两倍的高光时刻。
- 石油行业11年里，利润负增长了7次，但还没有出现过亏损，利润其实还不错。垄断行业的好处。
- 物流行业利润负增长这么多次不太好理解。

- 每年都上榜的有6家，全是银行。而且，工商银行每年都是冠军。
- 上榜的共有19家公司，银行占了11家，如果把平安和人寿算进去，金融占了13家。
- 4家资源性公司，三桶油加一个煤炭。
- 1家交通运输企业，中远海控。应该就是2021年疫情上的榜。世事无常，它还曾夺过亏损冠军。
- 1家高科技企业，中国移动。

In [97]:
for i in indu_res[3]:
    display(i)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

- 这63个从来没有亏损过的行业，平均增长率整体奇高，不太好理解。只能解释为大起大落。
- 这样就需要用夏普指数来算一下：平均增长率高但波动率小的才是好行业。

In [98]:
for i in indu_res[4]:
    display(i)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

- 夏普指数的观感比较符合直觉。
- 银行和酿酒行业确实是利润增长很不错的行业，但股价表现就不可同日而语了。

In [99]:
def win_code():
    # df = profit.merged.groupby(['code', 'year'], as_index=False)['profit'].mean()
    df = profit.merged.pivot_table(index='code', columns='year', values='profit', aggfunc='sum')
    wdf = df[df.min(1)>0]
    wdf_inc = wdf.pct_change(axis=1).iloc[:,1:]
    all_win = wdf_inc[wdf_inc.min(1)>0]
    stat = {'个股总数':len(df),'保持赢利个股':len(wdf),'保持利润增长': len(all_win)}

    win_bin = wdf_inc[wdf_inc < 0].count(axis=1).value_counts().sort_index()
    win_bin = win_bin.to_frame('行业数').reset_index(names= '负增长次数')
    plot_win_bin = profit.plot_ybar(win_bin, '出现负增长情况统计','.0f')

    count_gp = wdf_inc[wdf_inc < 0 ].count(axis=1).to_frame('counts').groupby('counts')
    def plot_indu_year(i, title):
        df = wdf_inc[wdf_inc.index.isin(count_gp.get_group(i).index) == True]
        df = df.melt(var_name='year', value_name='inc', ignore_index=False).reset_index()
        fig = alt.Chart(df).mark_line().encode(
            alt.X('year:O'),
            alt.Y('inc:Q'),
            alt.Color('indu').legend(orient='top')
        ).properties(
            title= title,
            width = 'container'
        )
        return fig

    def plot_bin(df, title, f='.0f'):
        # 第一列是属性，对第二列bin
        fig = alt.Chart(df, title= title, width='container').mark_bar().encode(
        alt.Y(f'{df.columns[1]}:Q').bin().axis(format=f),
        alt.X('count()').axis(title='数量'))
        return fig
    
    def plot_bin_most10(df, f, title):        
        # 数据分析做条形图
        # 取数据的前十和后十做条形图
        fig_bin = plot_bin(df, f'{title}分布', f)
        fig1 = profit.plot_hbar(df.head(10), f'{title}前十', f)
        fig2 = profit.plot_hbar(df.tail(10), f'{title}后十', f)
        return (fig_bin, fig1, fig2)
    
    def plot_mean():
        df = wdf_inc.mean(1).sort_values(ascending=0).to_frame('mean').reset_index()
        return plot_bin_most10(df, '.0%', '平均增长率')

    def plot_shapee():
        df = (wdf_inc.mean(1) / wdf_inc.std(1)).sort_values(ascending = 0).to_frame('shapee').reset_index()
        df = df.merge(profit.code[['code', 'name']])
        df = df[['name','shapee']]
        return plot_bin_most10(df, '.2f', '夏普指数')

    return (plot_win_bin, plot_indu_year, wdf_inc, plot_mean(), plot_shapee(), stat)

code_res = win_code()
